# Imports and Custom Variables

This section of the notebook imports necessary libraries and modules, and defines key variables used throughout the analysis. 

- **Imports**: Includes essential libraries like `sys` and custom modules such as `Gen_SPY_With_Indicators` and `generate_signals` for stock simulation and signal generation.
- **Custom Variables**:
    - `number_of_days`: Specifies the number of days for stock data simulation.
    - `intervalAmt`: Defines the interval for candlestick chart generation.

In [84]:
number_of_days = 2
intervalAmt = 15
import sys

# Simulated Stock Data Generation and Formatting for Candlestick Plot

The two cells below perform the following tasks:

1. **Simulated Stock Data Generation**:
    - The first cell uses the `simulate_stock` function from the `Gen_SPY_With_Indicators` module to generate simulated stock data for a specified number of days (`number_of_days`).
    - The simulation is based on the Black-Scholes equation, a mathematical model commonly used for financial markets to simulate stock price movements.
    - The output includes:
      - `simulated_data`: A DataFrame containing stock market data points such as `Timestamp`, `Close`, `Volume`, `Session`, `8EMA`, `VWAP`, `ORB_High`, `ORB_Low`, `PM_High`, `PM_Low`, `Yest_High`, `Yest_Low`, `Signal`, and `Stop_Loss`.
      - `yesterday_high` and `yesterday_low`: The high and low prices from the previous trading day.
    - The `generate_signals` function is then applied to the simulated data to add trading signals (e.g., "BUY CALL", "BUY PUT") and stop-loss levels.

2. **Data Formatting for Candlestick Plot**:
    - The second cell uses the `generate_candlestick` function from the `generate_candlestick_df` module to format the `simulated_data` for candlestick chart generation.
    - The function aggregates the data into intervals defined by `intervalAmt` (e.g., 15-minute intervals) and calculates the `Open`, `High`, `Low`, and `Close` prices for each interval.
    - The resulting `candlestick_data` DataFrame is indexed by `Timestamp` and includes columns such as `Open`, `High`, `Low`, `Close`, `Volume`, and `Day`.
    - This formatted data is essential for creating interactive candlestick plots, as it provides the necessary structure for visualizing stock price movements over time.

In [85]:
from Gen_SPY_With_Indicators import simulate_stock  # Importing simulation function
from generate_signals import generate_signals  # Importing signal generation function

# Simulate Stock Data for 2 Days
simulated_data, yesterday_high, yesterday_low = simulate_stock(number_of_days)

# Apply Signal Generation
simulated_data = generate_signals(simulated_data)

c:\Users\anton\OneDrive\Documents\GitHub\Financial-Engineering-Project\Gen_SPY_With_Indicators.py:108: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

c:\Users\anton\OneDrive\Documents\GitHub\Financial-Engineering-Project\Gen_SPY_With_Indicators.py:115: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

c:\Users\anton\OneDrive\Documents\GitHub\Financial-Engineering-Project\Gen_SPY_With_Indicators.py:108: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

c:\Users\anton\OneDrive\Documents\GitHub\Financial-Engineering-Project\Gen_SPY_With_Indicators.py:115: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [86]:
from generate_candlestick_df import generate_candlestick

# Generate candlestick data
candlestick_data = generate_candlestick(simulated_data, intervalAmt) # change this to change the interval on the candlestick chart
candlestick_data.set_index('Timestamp', inplace=True)

# Candlestick Chart with Indicators

This cell generates an interactive candlestick chart using Plotly, enriched with various technical indicators to provide insights into stock price movements. The chart includes:

- **Candlestick Data**: Visual representation of open, high, low, and close prices for each interval.
- **Indicators**:
    - **ORB High/Low**: Opening Range Breakout levels.
    - **Yesterday's High/Low**: Previous day's high and low prices.
    - **PM High/Low**: Pre-market high and low prices.
    - **8EMA**: Exponential Moving Average with a period of 8.
    - **VWAP**: Volume Weighted Average Price.
- **Session Data**:
    - Pre-market and regular market sessions are distinguished with separate line plots for their respective closing prices.

The chart is interactive, allowing users to hover over data points for detailed information and toggle between different views for better analysis.

In [87]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

def plot_candlestick_with_indicators(candlestick_data, simulated_data):
    # Create a figure with a single subplot
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.1, 
                        subplot_titles=("Candlestick Plot",))
    # Add candlestick traces
    fig.add_trace(go.Candlestick(
        x=candlestick_data.index,
        open=candlestick_data['Open'],
        high=candlestick_data['High'],
        low=candlestick_data['Low'],
        close=candlestick_data['Close'],
        name='Candlestick',
        opacity=1
    ), row=1, col=1)

    # Add indicators to the candlestick plot
    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['ORB_High'],
        mode='lines',
        name='ORB High',
        line=dict(color='green', dash='dash')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['ORB_Low'],
        mode='lines',
        name='ORB Low',
        line=dict(color='red', dash='dash')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['Yest_High'],
        mode='lines',
        name="Yesterday's High",
        line=dict(color='gray', dash='dashdot')
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['Yest_Low'],
        mode='lines',
        name="Yesterday's Low",
        line=dict(color='brown', dash='dashdot')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['PM_High'],
        mode='lines',
        name='PM High',
        line=dict(color='green', dash='dot')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['PM_Low'],
        mode='lines',
        name='PM Low',
        line=dict(color='red', dash='dot')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['8EMA'],
        mode='lines',
        name='8EMA',
        line=dict(color='orange')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['VWAP'],
        mode='lines',
        name='VWAP',
        line=dict(color='blue')
    ), row=1, col=1)

    # Add PM and Regular Market values
    fig.add_trace(go.Scatter(
        x=simulated_data[simulated_data['Session'] == 'PM']['Timestamp'],
        y=simulated_data[simulated_data['Session'] == 'PM']['Close'],
        mode='lines',
        name='Pre-Market Value',
        line=dict(color='black', dash='dot'),
        opacity=0.7
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=simulated_data[simulated_data['Session'] == 'Regular Market']['Timestamp'],
        y=simulated_data[simulated_data['Session'] == 'Regular Market']['Close'],
        mode='lines',
        name='Regular Market Value',
        line=dict(color='steelblue'),
        opacity=0.8
    ), row=1, col=1)

    # Update layout for better appearance
    fig.update_layout(
        title="Simulated Stock Price with Indicators",
        xaxis_title="Timestamp",
           yaxis_title="Price",
        legend_title="Legend",
        xaxis=dict(tickangle=45),
        template="seaborn",
        hovermode='x unified',
        height=800  # Make the graph taller
    )

    # Show the plot
    fig.show()

# Example usage
plot_candlestick_with_indicators(candlestick_data, simulated_data)